### Preparación de datos

In [ ]:
import pandas as pd
from pathlib import Path

# === RUTAS ===

ruta_acta =  Path(r"20251003/20251003/ADP_DTM_FACT.Acta.csv")
ruta_insumo =  Path(r"20251003/20251003/ADP_DTM_DIM.Insumo.csv")
ruta_proyectos =  Path(r"20251003/20251003/ADP_DTM_DIM.Proyecto.csv")
ruta_capitulos = Path(r"20251003/20251003/ADP_DTM_DIM.CapituloPresupuesto.csv")
ruta_proyeccion =  Path(r"20251003/20251003/ADP_DTM_FACT.Proyeccion.csv")
ruta_items =  Path(r"20251003/20251003/ADP_DTM_DIM.Items.csv")
ruta_macro = Path(r"20251003/20251003/proyectos_macroproyectos.csv")
tabla_macro = pd.read_csv(ruta_macro)


In [17]:
# === CARGA ===
tabla_proyeccion = pd.read_csv(ruta_proyeccion)
tabla_items = pd.read_csv(ruta_items)
tabla_proyectos = pd.read_csv(ruta_proyectos)
tabla_capitulos = pd.read_csv(ruta_capitulos)
tabla_insumos = pd.read_csv(ruta_insumo)
tabla_macro = pd.read_csv(ruta_macro)

C:\Users\aleja\AppData\Local\Temp\ipykernel_33476\1549343039.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  tabla_items = pd.read_csv(ruta_items)


In [9]:
tabla_base = tabla_proyeccion.copy()

# Merges previos
tabla_1 = pd.merge(tabla_base, tabla_proyectos, on="SkIdProyecto", how="left")
tabla_2 = pd.merge(tabla_1, tabla_capitulos, on="SkIdCapitulo", how="left")

# Asegura unicidad en Items
tabla_items_unica = tabla_items.drop_duplicates(subset=["SkIdItems"], keep="first")
tabla_3 = pd.merge(
    tabla_2, tabla_items_unica,
    on="SkIdItems", how="left",
    suffixes=("", "_item")   # control de sufijos para items
)

# 🔧 FIX insumos: prefijar y garantizar clave única
tabla_insumos_unica = tabla_insumos.drop_duplicates(subset=["SkIdInsumo"], keep="first").copy()
cols_no_clave_insumo = [c for c in tabla_insumos_unica.columns if c != "SkIdInsumo"]
tabla_insumos_pref = tabla_insumos_unica.rename(
    columns={c: f"Insumo_{c}" for c in cols_no_clave_insumo}
)

# (Opcional) Debug rápido de posibles overlaps antes del merge final
overlap_prev = set(tabla_3.columns).intersection(set(tabla_insumos_pref.columns))
print(f"[DEBUG] Overlap antes de merge con insumos (debería ser solo 'SkIdInsumo'): {overlap_prev}")

# Merge final (sin sufijos porque ya prefijamos)
tabla_4 = pd.merge(
    tabla_3, tabla_insumos_pref,
    on="SkIdInsumo", how="left",
    validate="many_to_one"
)

print(f"[DEBUG] OK. Forma final: {tabla_4.shape}")

tabla_4.to_csv(
    "tabla_looker.csv",
    index=False,
    encoding="utf-8",
    sep=","                
)

[DEBUG] Overlap antes de merge con insumos (debería ser solo 'SkIdInsumo'): {'SkIdInsumo'}
[DEBUG] OK. Forma final: (273450, 107)


#### New Join

In [18]:
# Alinear columnas
if "Proyectos" in tabla_macro.columns:
    tabla_macro = tabla_macro.rename(columns={"Proyectos": "Nombre Proyecto"})
print("[DEBUG] Columnas alineadas.")

# Verificar coincidencias
interseccion = set(tabla_proyectos["Nombre Proyecto"]).intersection(set(tabla_macro["Proyecto"]))
print(f"[DEBUG] Coincidencias exactas entre tablas: {len(interseccion)}")

# === Merge de proyectos con macroproyectos ===
tabla_proyectos_macro = pd.merge(
    tabla_proyectos,
    tabla_macro,
    left_on="Nombre Proyecto",
    right_on="Proyecto",
    how="left",
)

print(f"[DEBUG] Merge proyectos + macroproyectos: {tabla_proyectos_macro.shape}")

# === PIPELINE COMPLETO ===
tabla_base = tabla_proyeccion.copy()

tabla_1 = pd.merge(tabla_base, tabla_proyectos_macro, on="SkIdProyecto", how="left")
tabla_2 = pd.merge(tabla_1, tabla_capitulos, on="SkIdCapitulo", how="left")

# Items
tabla_items_unica = tabla_items.drop_duplicates(subset=["SkIdItems"], keep="first")
tabla_3 = pd.merge(tabla_2, tabla_items_unica, on="SkIdItems", how="left", suffixes=("", "_item"))

# Insumos
tabla_insumos_unica = tabla_insumos.drop_duplicates(subset=["SkIdInsumo"], keep="first").copy()
cols_no_clave = [c for c in tabla_insumos_unica.columns if c != "SkIdInsumo"]
tabla_insumos_pref = tabla_insumos_unica.rename(columns={c: f"Insumo_{c}" for c in cols_no_clave})

tabla_final = pd.merge(tabla_3, tabla_insumos_pref, on="SkIdInsumo", how="left")
print(f"[DEBUG] OK. Forma final: {tabla_final.shape}")

# === Exportar resultados ===
tabla_final.to_csv("tabla_looker.csv", index=False, encoding="utf-8", sep=",")
tabla_final.to_parquet("tabla_looker.parquet", index=False)

print("[DEBUG] Exportado CSV y Parquet correctamente.")

# Vista rápida de verificación
display(tabla_final[["Nombre Proyecto", "Macroproyecto"]].head(10))

[DEBUG] Columnas alineadas.
[DEBUG] Coincidencias exactas entre tablas: 0
[DEBUG] Merge proyectos + macroproyectos: (85, 43)
[DEBUG] OK. Forma final: (273450, 113)
[DEBUG] Exportado CSV y Parquet correctamente.


,Nombre Proyecto,Macroproyecto
0,URBAN PLAZA,NaN
1,URBAN PLAZA,NaN
2,URBAN PLAZA,NaN
3,URBAN PLAZA,NaN
4,URBAN PLAZA,NaN
5,URBAN PLAZA,NaN
6,URBAN PLAZA,NaN
7,URBAN PLAZA,NaN
8,URBAN PLAZA,NaN
9,URBAN PLAZA,NaN


In [19]:
# === SELECCIÓN DE COLUMNAS ===
columnas_finales = [
    "SkIdProyecto", "SkIdCapitulo", "SkIdItems", "SkIdInsumo",
    "Nombre Proyecto", "Capitulo Descripcion", "Item Descripcion",
    "Insumo_Insumo Descripcion", "Insumo_Agrupacion Descripcion",
    "SkIdFecha Real", "Cantidad", "Valor Unitario", "Valor Total",
    "Insumo_Valor Unitario", "Insumo_Valor Neto", "Insumo_Fecha Creacion",
    "Cantidad Item", "Macroproyecto", "Insumo_Fecha Modificacion",
    "Fecha De Elaboracion", "Fecha De Inicio", "Fecha De Finalización",
    "SkIdFecha", "Capitulo Numero", "Cantidad_Item"
]

# Filtrar solo las columnas que existan realmente (por seguridad)
columnas_existentes = [col for col in columnas_finales if col in tabla_final.columns]
tabla_looker = tabla_final[columnas_existentes].copy()

print(f"[DEBUG] Columnas seleccionadas: {len(columnas_existentes)} / {len(columnas_finales)}")
print(f"[DEBUG] Forma final de tabla_looker: {tabla_looker.shape}")
display(tabla_looker.head(5))


[DEBUG] Columnas seleccionadas: 23 / 25
[DEBUG] Forma final de tabla_looker: (273450, 23)


,SkIdProyecto,SkIdCapitulo,SkIdItems,SkIdInsumo,Nombre Proyecto,Capitulo Descripcion,Item Descripcion,Insumo_Insumo Descripcion,Insumo_Agrupacion Descripcion,SkIdFecha Real,...,Insumo_Valor Unitario,Insumo_Valor Neto,Insumo_Fecha Creacion,Cantidad Item,Macroproyecto,Insumo_Fecha Modificacion,Fecha De Elaboracion,Fecha De Inicio,SkIdFecha,Capitulo Numero
0,1005,1005100,1006583,100279,URBAN PLAZA,CIMENTACION,Pilote preexcavado d=80cm,agua carrotanque,Campamento y provisionales,20110929,...,7327.59,8719.8321,06/12/2010,2.00,NaN,13/01/2011,28/03/2011,25/01/2011,20110929,2
1,1005,1005100,1006583,1002087,URBAN PLAZA,CIMENTACION,Pilote preexcavado d=80cm,concreto tremie 3000 psi grava común,Concretos Especiales,20110929,...,263115.84,313107.8496,06/12/2010,2.00,NaN,NaN,28/03/2011,25/01/2011,20110929,2
2,1005,1005100,1006583,1006063,URBAN PLAZA,CIMENTACION,Pilote preexcavado d=80cm,subcontrato pilote preexcavado d=0.80m,Subcontratos Preliminares Cimentación y Estruc...,20110929,...,40250.00,40250.0000,06/12/2010,2.00,NaN,27/05/2011,28/03/2011,25/01/2011,20110929,2
3,1005,1005100,1007643,100279,URBAN PLAZA,CIMENTACION,Pantalla preexcavada,agua carrotanque,Campamento y provisionales,20110930,...,7327.59,8719.8321,06/12/2010,32.28,NaN,13/01/2011,28/03/2011,25/01/2011,20110930,2
4,1005,1005100,1007643,1002087,URBAN PLAZA,CIMENTACION,Pantalla preexcavada,concreto tremie 3000 psi grava común,Concretos Especiales,20110930,...,263115.84,313107.8496,06/12/2010,32.28,NaN,NaN,28/03/2011,25/01/2011,20110930,2


#### Debug

In [15]:
# === REVISAR POSIBLE MERGE ENTRE tabla_proyectos Y proyectos_macroproyectos.csv ===

import pandas as pd

# Cargar la tabla externa
ruta_macro = Path(r"20251003/20251003/proyectos_macroproyectos.csv")
tabla_macro = pd.read_csv(ruta_macro)

print(f"[DEBUG] Cargadas {len(tabla_macro)} filas desde {ruta_macro}")
print(f"[DEBUG] Columnas detectadas: {tabla_macro.columns.tolist()}")

# Renombrar para alinear con tabla_proyectos
if "Proyectos" in tabla_macro.columns:
    tabla_macro = tabla_macro.rename(columns={"Proyectos": "Nombre Proyecto"})
    print("[DEBUG] Columna 'Proyectos' renombrada a 'Nombre Proyecto' para coincidir con tabla_proyectos.")

# Verificar columnas clave
if "Nombre Proyecto" not in tabla_proyectos.columns:
    print("[DEBUG] ERROR: No existe la columna 'Nombre Proyecto' en tabla_proyectos.")
else:
    print("[DEBUG] OK: 'Nombre Proyecto' presente en tabla 1.")

# Revisar unicidad y correspondencia
unic_proyectos = tabla_proyectos["Nombre Proyecto"].nunique()
unic_macro = tabla_macro["Proyecto"].nunique()
print(f"[DEBUG] Unicidad: tabla_proyectos={unic_proyectos} | tabla_macro={unic_macro}")

# Evaluar intersección y posibles pérdidas
interseccion = set(tabla_proyectos["Nombre Proyecto"]).intersection(set(tabla_macro["Proyecto"]))
faltantes_en_macro = set(tabla_proyectos["Nombre Proyecto"]) - set(tabla_macro["Proyecto"])
faltantes_en_proyectos = set(tabla_macro["Proyecto"]) - set(tabla_proyectos["Nombre Proyecto"])

print(f"[DEBUG] Coincidencias exactas: {len(interseccion)}")
print(f"[DEBUG] Faltan en macroproyectos.csv: {len(faltantes_en_macro)}")
print(f"[DEBUG] Faltan en tabla_proyectos: {len(faltantes_en_proyectos)}")

# Muestra rápida de diferencias
if len(faltantes_en_macro) > 0:
    print("[DEBUG] Ejemplo de proyectos presentes en tabla_proyectos pero no en macroproyectos:")
    print(list(faltantes_en_macro)[:5])

# Ver si hay duplicados por proyecto
duplicados_macro = tabla_macro["Proyecto"].duplicated().sum()
duplicados_proyectos = tabla_proyectos["Nombre Proyecto"].duplicated().sum()
print(f"[DEBUG] Duplicados: macro={duplicados_macro}, proyectos={duplicados_proyectos}")

# Recomendación de tipo de merge
if duplicados_macro == 0 and duplicados_proyectos == 0:
    print("[DEBUG] MERGE RECOMENDADO: one-to-one (merge tipo LEFT o INNER según necesidad).")
elif duplicados_macro > 0 and duplicados_proyectos == 0:
    print("[DEBUG] MERGE RECOMENDADO: many-to-one (LEFT con validate='many_to_one').")
elif duplicados_macro == 0 and duplicados_proyectos > 0:
    print("[DEBUG] MERGE RECOMENDADO: one-to-many (LEFT con validate='one_to_many').")
else:
    print("[DEBUG] MERGE RECOMENDADO: many-to-many (revisar duplicados antes de proceder).")


[DEBUG] Cargadas 74 filas desde 20251003\20251003\proyectos_macroproyectos.csv
[DEBUG] Columnas detectadas: ['Proyecto', 'Sucursal', 'Centro de costo', 'Estado', 'Tipo de proyecto', 'Macroproyecto']
[DEBUG] OK: 'Nombre Proyecto' presente en tabla 1.
[DEBUG] Unicidad: tabla_proyectos=85 | tabla_macro=74
[DEBUG] Coincidencias exactas: 0
[DEBUG] Faltan en macroproyectos.csv: 85
[DEBUG] Faltan en tabla_proyectos: 74
[DEBUG] Ejemplo de proyectos presentes en tabla_proyectos pero no en macroproyectos:
['Samanes Alta Vista - Etapa 2', 'Centro Cívico Uniandes', 'Connecta 80', 'Edificio Automercol', 'Edif Bloque C Uniandes']
[DEBUG] Duplicados: macro=0, proyectos=0
[DEBUG] MERGE RECOMENDADO: one-to-one (merge tipo LEFT o INNER según necesidad).


# CONCATENAR DESCRIPCIONES JERÁRQUICAS Y MOSTRAR MUESTRA

In [21]:
# === CONCATENAR DESCRIPCIONES JERÁRQUICAS Y MOSTRAR MUESTRA ===

import pandas as pd

def construir_descripcion_jerarquica(fila: pd.Series,
                                     columnas_a_concatenar: list[str],
                                     separador: str = " - ") -> str:
    """
    Construye una cadena 'jerárquica' uniendo, en orden, los valores no vacíos
    de las columnas indicadas. Omite NaN y strings vacíos.
    """
    partes_limpias: list[str] = []
    for nombre_columna in columnas_a_concatenar:
        valor = fila.get(nombre_columna, "")
        if pd.isna(valor):
            continue
        valor_str = str(valor).strip()
        if valor_str:
            partes_limpias.append(valor_str)
    return separador.join(partes_limpias)

# Columnas que deseamos concatenar, en orden
columnas_objetivo: list[str] = [
    "Macroproyecto",
    "Nombre Proyecto",
    "Capitulo Descripcion",
    "Item Descripcion",
    "Insumo_Insumo Descripcion",
]

# Verificación de existencia de columnas (seguimos con las que existan)
columnas_que_existen = [c for c in columnas_objetivo if c in tabla_looker.columns]
columnas_que_faltan = [c for c in columnas_objetivo if c not in tabla_looker.columns]

print(f"[DEBUG] Columnas objetivo: {columnas_objetivo}")
print(f"[DEBUG] Columnas encontradas en 'tabla_looker': {columnas_que_existen}")
if columnas_que_faltan:
    print(f"[DEBUG] ADVERTENCIA: Faltan estas columnas y se omitirán: {columnas_que_faltan}")

# Construir la nueva columna con la concatenación
nombre_columna_salida = "Descripcion_Jerarquica"
tabla_looker[nombre_columna_salida] = tabla_looker.apply(
    construir_descripcion_jerarquica,
    axis=1,
    columnas_a_concatenar=columnas_que_existen,
    separador=" - "
)

# Mostrar una muestra pequeña y legible
tamano_muestra: int = 100
columnas_para_mostrar = [
   nombre_columna_salida
]
columnas_presentes = [c for c in columnas_para_mostrar if c in tabla_looker.columns]

print(f"[DEBUG] Tamaño de la tabla con la nueva columna: {tabla_looker.shape}")
print(f"[DEBUG] Mostrando las primeras {tamano_muestra} filas con columnas: {columnas_presentes}")

muestra_pequena = tabla_looker[columnas_presentes].head(tamano_muestra)
display(muestra_pequena)  # en Jupyter/Colab; si prefieres, usa print(muestra_pequena.to_string(index=False))


[DEBUG] Columnas objetivo: ['Macroproyecto', 'Nombre Proyecto', 'Capitulo Descripcion', 'Item Descripcion', 'Insumo_Insumo Descripcion']
[DEBUG] Columnas encontradas en 'tabla_looker': ['Macroproyecto', 'Nombre Proyecto', 'Capitulo Descripcion', 'Item Descripcion', 'Insumo_Insumo Descripcion']
[DEBUG] Tamaño de la tabla con la nueva columna: (273450, 24)
[DEBUG] Mostrando las primeras 100 filas con columnas: ['Descripcion_Jerarquica']


,Descripcion_Jerarquica
0,URBAN PLAZA - CIMENTACION - Pilote preexcavado...
1,URBAN PLAZA - CIMENTACION - Pilote preexcavado...
2,URBAN PLAZA - CIMENTACION - Pilote preexcavado...
3,URBAN PLAZA - CIMENTACION - Pantalla preexcava...
4,URBAN PLAZA - CIMENTACION - Pantalla preexcava...
...,...
95,URBAN PLAZA - ESTRUCTURA - Escalera en concret...
96,URBAN PLAZA - ESTRUCTURA - Escalera en concret...
97,URBAN PLAZA - ESTRUCTURA - Escalera en concret...
98,URBAN PLAZA - ESTRUCTURA - Escalera en concret...


In [26]:
# === GUARDAR TABLA EN PARQUET ===
import os

# Ruta y nombre de salida (usa mismo nombre base que el CSV)
nombre_archivo_parquet = "tabla_looker.parquet"
ruta_salida_parquet = os.path.join(os.getcwd(), nombre_archivo_parquet)

# Guardar en formato Parquet
try:
    tabla_looker.to_parquet(ruta_salida_parquet, index=False)
    print(f"[DEBUG] Archivo guardado correctamente en formato Parquet: {ruta_salida_parquet}")
    print(f"[DEBUG] Tamaño de la tabla guardada: {tabla_looker.shape}")
except Exception as error:
    print(f"[DEBUG] ERROR al guardar el archivo Parquet: {error}")


[DEBUG] Archivo guardado correctamente en formato Parquet: c:\Users\aleja\Documents\Ingenieria Estadistica\Asignaturas2025B\arpro1\Base de Datos ARPRO\tabla_looker.parquet
[DEBUG] Tamaño de la tabla guardada: (273450, 24)
